## Libraries:

In [298]:
from pymongo import MongoClient
import json
import pandas as pd
from pprint import pprint


# Global Variables:

In [312]:
dbCreds = getDBCreds()
client = connectToDB(dbCreds)

# Helper Functions

### DB Helper Functions:

In [4]:
def getDBCreds():
    credentials = json.load(open('../credentials.json'))
    dbCreds = credentials['database']
    return dbCreds

In [6]:
def connectToDB(dbCreds):
    cosmosConnString = "mongodb://" + dbCreds['username'] + ":" + dbCreds['password'] + "@" + dbCreds['host'] + ":" + str(dbCreds['port']) + "/?ssl=true&replicaSet=globaldb"
    client = MongoClient(cosmosConnString)
    return client

### Disease Helper Funtions:

In [8]:
def getLiterature(client):
    db = client["admin"]
    literatureCollection = db["literatures"]
    return literatureCollection

In [108]:
def getSymptomList(client):
    db = client['admin']
    symptomCollection = db['symptoms']
    symptomListCursor = symptomCollection.find(filter={})
    symptomList = []
    for element in symptomListCursor:
        symptomList.append(element['name'])
    return symptomList

### The following code segment showcases how pandas can be used for more efficient symptom analysis

In [1]:
# symptomList = getSymptomList(client)
# dfColumns = ["True"]

# symptomDf = pd.DataFrame(index=testSymptomList, columns=dfColumns)

# symptomDf = testDf.fillna(0)
# targetSymptoms = []
# for index,row in testDf.iterrows():
#     if(row.iloc[0] == 1):
#         print("Adding symptom to targetSymptom list")
#         print(row.name)
#         ***modify df here to generate a sparse matrix***
#         validSymptoms.append(row.name)

### **%%%End of Sample Code Block%%%**

In [310]:
def diagnoseDisease(client, symptomList):
    flag = False
    targetSymptoms = []
    nonTargetSymptoms = []
    symptomListLength = len(symptomList)
    count = 0
    while(flag !=True):
        for symptom in symptomList:
            ans = input("Do you have " + symptom + "? \n")
            if(ans == 'yes'):
                targetSymptoms.append(symptom)
                queryResult = getDiseaseList(targetSymptoms, nonTargetSymptoms, len(targetSymptoms), len(nonTargetSymptoms))
                if(queryResult['count']==1):
                    flag = True
                    response = "The diagnosed disease is " + queryResult['diseases'][0]
                    return(response)
                if(queryResult['count']==0):
                    flag = True
                    response = "It seems the symptoms do not match anything from the database"
                    return(response)
            else:
                nonTargetSymptoms.append(symptom)
                queryResult = getDiseaseList(targetSymptoms, nonTargetSymptoms, len(targetSymptoms), len(nonTargetSymptoms))
                if(queryResult['count']==0):
                    flag = True
                    response = "It seems the symptoms do not match anything from the database"
                    return(response)

In [302]:
def getDiseaseList(client, symptoms, nonTargetSymptoms, targetSymptomCount, nonTargetSymptomCount):
    literatureCollection = getLiterature(client)
    if(targetSymptomCount==0):
        diseaseList = literatureCollection.find(filter={"symptoms" : {"$nin" : nonTargetSymptoms}})
    else:
        diseaseList = literatureCollection.find(filter={"symptoms" : {"$all" : symptoms, "$nin" : nonTargetSymptoms}})
    count = 0
    diseases = []
    for disease in diseaseList:
        count = count + 1
        diseases.append(disease['title'])
    return({"diseases":diseases,"count":count})

In [8]:
def findSymptoms(diseaseName, literatureCollection):
    result = literatureCollection.find_one(filter={"title" : diseaseName})
    symptoms = ""
    for symptom in result["symptoms"]:
        symptoms = symptoms + ", " + symptom
    response = "The symptoms of " + diseaseName + " are " + symptoms
    return response

In [9]:
def findDescription(diseaseName, literatureCollection):
    result = literatureCollection.find_one(filter={"title" : diseaseName})
    diseaseDescription = result["content"]
    return diseaseDescription

### Things to export:

1. findDescription
2. findSymptoms
3. diagnoseDisease